In [ ]:

import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50

In [ ]:
imageSize=224
test_dir = ('../input/oct2017/OCT2017 /test/')

# ['DME', 'CNV', 'NORMAL', '.DS_Store', 'DRUSEN']
from tqdm import tqdm
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['NORMAL']:
                label = 0
            elif folderName in ['CNV']:
                label = 1
            elif folderName in ['DME']:
                label = 2
            elif folderName in ['DRUSEN']:
                label = 3
            else:
                label = 4
            for image_filename in tqdm(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 1))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X) # Keras only accepts data as numpy arrays 
    y = np.asarray(y)
    return X,y
#X_test, y_test = get_data(test_dir) # Un-comment to use full dataset: Step 1 of 2
X_test, y_test= get_data(test_dir)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.2) # comment this ligne  to use full dataset: Step 2 of 2

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(y_train, num_classes = 4)
y_testHot = to_categorical(y_test, num_classes = 4)


In [ ]:
x_train, y_train = next(training_set)

#valid_X,valid_Y=next(validation_set)
xtest,ystest=next(validation_set)

In [ ]:
m_samples = x_train.shape[0]
m_samplesTest = xtest.shape[0]
X_train2 = x_train.reshape(m_samples, -1)
X_test2 = xtest.reshape(m_samplesTest, -1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score  

RandomForestClassifier=RandomForestClassifier(n_estimators=300)

RandomForestClassifier.fit(X_train2,y_train)


In [ ]:
pred=RandomForestClassifier.predict(X_test2)

Accuracy=RandomForestClassifier.score(X_test2,ystest)
print('The accuracy of Random Forest classifier is : ',Accuracy)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ystest, pred))


In [ ]:
from sklearn.metrics import f1_score
f1_score(ystest, pred, average='macro')  

In [ ]:
from sklearn.metrics import precision_score
precision_score(ystest, pred, average='weighted')  

In [ ]:
from sklearn.metrics import recall_score
recall_score(ystest, pred, average='weighted')  

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
clf = svm.SVC(gamma='scale')
#fit to the trainin data
clf.fit(X_train2,y_train)



In [ ]:
y_predSVM = classifierSVM.predict(X_test2)
print ('SVM  ACCURACY : ',accuracy_score(y_test, y_predSVM))


In [ ]:
 

print('SVM Precision ',precision_score(y_test, y_predSVM, average='weighted'))  
print('SVM Precision ',recall_score(y_test, y_predSVM, average='weighted')  )
print('SVM F1 score ',f1_score(y_test, y_predSVM, average='weighted') ) 

In [ ]:
from xgboost import XGBClassifier
classifierG = XGBClassifier()
classifierG.fit(X_train2,y_train)
# Predicting the Test set results
y_predXG = classifierG.predict(X_test2)


In [ ]:
print ('XGBOOST ACCURACY : ',accuracy_score(y_test, y_predXG))


In [ ]:
 

print('XGBOOST  Precision ',precision_score(y_test, y_predXG, average='weighted'))  
print('XGBOOST recall ',recall_score(y_test, y_predXG, average='weighted')  )
print('XGBOOST F1 score ',f1_score(y_test, y_predXG, average='weighted') ) 

In [ ]:
from sklearn import tree

model = tree.DecisionTreeClassifier()

# Train Decision Tree Classifer


In [ ]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 90000))
# classifier.add(Dropout(p = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(p = 0.1))

# Adding the output layer
classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(X_train, y_trainHot, batch_size = 10, epochs = 100)


In [ ]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (4, 4)))
classifier.add(Dropout(0.1))
classifier.add(Conv2D(512, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (3, 3)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 4, activation = 'sigmoid'))
#classifier.summary()

In [ ]:
# Compiling the CNN

#classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
  rescale = 1./255,
  validation_split=0.2
  )
training_set = train_datagen.flow_from_directory('../input/oct2017/OCT2017 /train',
  target_size = (224, 224),
  batch_size = 80,
  classes=["DME","CNV","NORMAL","DRUSEN"],
  subset='training')
 
validation_set = train_datagen.flow_from_directory('../input/oct2017/OCT2017 /train',
  target_size = (224, 224),
  batch_size = 80,
  subset='validation',
  classes=["DME","CNV","NORMAL","DRUSEN"])

test_set = train_datagen.flow_from_directory('../input/oct2017/OCT2017 /test',
  target_size = (224, 224),
  batch_size = 968,
  classes=["DME","CNV","NORMAL","DRUSEN"])

In [ ]:
x_train, y_train = next(training_set)

valid_X,valid_Y=next(validation_set)
xtest,ystest=next(test_set)

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
 
class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)


def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')
    plt.ylabel('Accuracy')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')
map_characters1 = {0: 'Normal', 1: 'CNV', 2: 'DME', 3: 'DRUSEN'}


In [ ]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

 # creating checkpoint
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list1 = [checkpoint]
callbacks_list = callbacks_list1+[keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1)]


In [ ]:
classifier.fit(X_train, y_trainHot,validation_data=(X_test,y_testHot) ,batch_size = 5, epochs = 10)


In [ ]:
RetinalClasses = ["CNV","DME","DRUSEN","NORMAL"]

PredictPathology = classifier.predict(X_test)


figure = plt.figure(figsize=(20, 8))

for i, index in enumerate(np.random.choice(X_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(X_test[index]))
    predict_index = np.argmax(PredictPathology[index])
    true_index = np.argmax(y_testHot[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(RetinalClasses[predict_index], 
                                  RetinalClasses[true_index]),
                                  color=("green" if predict_index == true_index else "red"))
plt.show()


In [ ]:
# training model
history = classifier.fit_generator(training_set, 
                                  steps_per_epoch = 66788/80,
                                  validation_data = (valid_X,valid_Y),
                                  validation_steps=208,
                                  epochs = 20,
                                  shuffle=True, 
                                  callbacks = callbacks_list+[MetricsCheckpoint('logs')])


In [ ]:
labels = map_characters1

# Evaluate model
score = classifier.evaluate(xtest,ystest, verbose=0)
print('\n  CNN architecture   - accuracy ON TEST SET :', score[1], '\n')
y_pred = classifier.predict(xtest)
print('\n', sklearn.metrics.classification_report(np.where(ystest > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 
Y_pred_classes = np.argmax(y_pred,axis = 1) 
Y_true = np.argmax(ystest,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
#plotKerasLearningCurve()
#plt.show()
#plot_learning_curve(history)
#plt.show()
plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
plt.show()


In [ ]:
classifier.save('trained.h5')

In [ ]:
RetinalClasses = ["CNV","DME","DRUSEN","NORMAL"]

PredictPathology = classifier.predict(xtest)



In [ ]:

figure = plt.figure(figsize=(20, 8))

for i, index in enumerate(np.random.choice(xtest.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(xtest[index]))
    predict_index = np.argmax(PredictPathology[index])
    true_index = np.argmax(ystest[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(RetinalClasses[predict_index], 
                                  RetinalClasses[true_index]),
                                  color=("green" if predict_index == true_index else "red"))
plt.show()
